# Level Break Out

### Load the data

In [2]:
# Inital Imports
import pandas as pd
import numpy as np
from scipy import stats
import pandas_ta as ta
import plotly.graph_objects as go
import yfinance as yf


In [3]:
# Specify the stock symbol (NVIDIA in this case) and the date range
stock_symbol = "AAPL"
start_date = "2023-01-18"
end_date = "2024-01-18"

# Fetch stock data from Yahoo Finance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Create DataFrame from the stock data
df = pd.DataFrame(stock_data)

# Filter rows with non-zero volume
#df = df[df['Volume'] != 0]
df.reset_index(inplace=True)  # Resetting index to get timestamp as a column


[*********************100%%**********************]  1 of 1 completed


In [4]:
# Create function isPivot to detect highs and lows
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, -1 if pivot low, 2 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0

    pivotHigh = 1
    pivotLow = -1
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 2
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [5]:
# Run isPivot function and store high and low points in new column with a window of +-5 candles 
window=3
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [6]:
# Review Dataframe
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,isPivot
0,2023-01-18,136.820007,138.610001,135.029999,135.210007,134.458450,69672800,0
1,2023-01-19,134.080002,136.250000,133.770004,135.270004,134.518112,58280400,0
2,2023-01-20,135.279999,138.020004,134.220001,137.869995,137.103668,80223600,0
3,2023-01-23,138.119995,143.320007,137.899994,141.110001,140.325638,81760300,0
4,2023-01-24,140.309998,143.160004,140.300003,142.529999,141.737747,66435100,0


In [7]:
# Create pointpos function to help visualize isPivot
def pointpos(x):
    if x['isPivot']==-1:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [8]:
# Plot candles and pivot points
dfpl = df
#dfpl = df[df['Date'].dt.year == 2020]
#dfpl = df[(df['Date'].dt.year >= 2020) & (df['Date'].dt.year <= 2023)]
fig = go.Figure(data=[go.Candlestick(x=dfpl['Date'],
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl['Date'], y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(
    title_text="AAPL Chart with Pivot Points",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False
)
fig.show()

In [9]:
# Drop pointpos columns
df.drop(columns='pointpos', inplace=True)
# Rename isPivot column to signal
df.rename(columns={'isPivot': 'signal'}, inplace=True)
# Review dataframe
df

,Date,Open,High,Low,Close,Adj Close,Volume,signal
0,2023-01-18,136.820007,138.610001,135.029999,135.210007,134.458450,69672800,0
1,2023-01-19,134.080002,136.250000,133.770004,135.270004,134.518112,58280400,0
2,2023-01-20,135.279999,138.020004,134.220001,137.869995,137.103668,80223600,0
3,2023-01-23,138.119995,143.320007,137.899994,141.110001,140.325638,81760300,0
4,2023-01-24,140.309998,143.160004,140.300003,142.529999,141.737747,66435100,0
...,...,...,...,...,...,...,...,...
246,2024-01-10,184.350006,186.399994,183.919998,186.190002,186.190002,46792900,0
247,2024-01-11,186.539993,187.050003,183.619995,185.589996,185.589996,49128400,1
248,2024-01-12,186.059998,186.740005,185.190002,185.919998,185.919998,40444700,0
249,2024-01-16,182.160004,184.259995,180.929993,183.630005,183.630005,65603000,0


In [10]:
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume,signal
Date,,,,,,,
2023-01-18,136.820007,138.610001,135.029999,135.210007,134.458450,69672800,0
2023-01-19,134.080002,136.250000,133.770004,135.270004,134.518112,58280400,0
2023-01-20,135.279999,138.020004,134.220001,137.869995,137.103668,80223600,0
2023-01-23,138.119995,143.320007,137.899994,141.110001,140.325638,81760300,0
2023-01-24,140.309998,143.160004,140.300003,142.529999,141.737747,66435100,0


In [11]:
# Export to csv
df.to_csv('signals_appl.csv', index=True)
